In [1]:
!pip install kagglehub


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:

import kagglehub

# Download latest version
path = kagglehub.dataset_download("saksham07gupta/road-obstacledebris-detection")

print("Path to dataset files:", path)

Path to dataset files: /home/imadb/.cache/kagglehub/datasets/saksham07gupta/road-obstacledebris-detection/versions/2


In [3]:
import os

base_path = "/home/imadb/.cache/kagglehub/datasets/saksham07gupta/road-obstacledebris-detection/versions/2"
os.listdir(base_path)



['final_dataset', 'data_fixed.yaml', 'data.yaml']

In [4]:
os.listdir(base_path + "/final_dataset/final_dataset")
os.listdir(base_path + "/final_dataset/final_dataset/train")



['images', 'labels.cache', 'labels']

In [5]:
with open(base_path + "/data.yaml", "r") as f:
    print(f.read())


train: ../train/images
val: ../valid/images
test: ../test/images

nc: 3
names:
  0: garbage
  1: road_block
  2: fallen_tree



In [6]:
base_path = "/home/imadb/.cache/kagglehub/datasets/saksham07gupta/road-obstacledebris-detection/versions/2"

fixed_yaml = f"""path: {base_path}/final_dataset/final_dataset
train: train/images
val: valid/images
test: test/images

nc: 3
names:
  0: garbage
  1: road_block
  2: fallen_tree
"""

with open(base_path + "/data_fixed.yaml", "w") as f:
    f.write(fixed_yaml)

print(" écrit :", base_path + "/data_fixed.yaml")


 écrit : /home/imadb/.cache/kagglehub/datasets/saksham07gupta/road-obstacledebris-detection/versions/2/data_fixed.yaml


In [7]:
import os

base = "/home/imadb/.cache/kagglehub/datasets/saksham07gupta/road-obstacledebris-detection/versions/2/final_dataset/final_dataset"

print("train images:", len(os.listdir(base + "/train/images")))
print("train labels:", len(os.listdir(base + "/train/labels")))

print("valid images:", len(os.listdir(base + "/valid/images")))
print("valid labels:", len(os.listdir(base + "/valid/labels")))

print("test images:", len(os.listdir(base + "/test/images")))
print("test labels:", len(os.listdir(base + "/test/labels")))


train images: 3766
train labels: 3766
valid images: 722
valid labels: 722
test images: 695
test labels: 695


In [10]:
from ultralytics import YOLO
import cv2
import os

#  Charger le modèle entraîné
model = YOLO("/home/imadb/runs/detect/road_obstacle_yolov8l/weights/best.pt")

# Image à tester (une image du dossier test)
img_path = "/home/imadb/Ghiles/road_debris/154_jpeg.rf.ee4fd023c091b6443febf288fb3f39b7.jpg"

#  Inference (sans dessin automatique)
results = model(img_path, verbose=False)[0]

print("=== Détections ===")
for box in results.boxes:
    cls = int(box.cls)
    name = results.names[cls]
    conf = float(box.conf)
    print(f"- {name} ({conf:.2f})")

#  Charger l'image originale
img = cv2.imread(img_path)

#  Dessiner les boîtes
for box in results.boxes:
    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
    cls = int(box.cls)
    name = results.names[cls]
    conf = float(box.conf)

    label = f"{name} {conf:.2f}"
    color = (0, 255, 0)  # vert

    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
    cv2.putText(
        img,
        label,
        (x1, y1 - 5),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        color,
        2,
        cv2.LINE_AA
    )

#  Sauvegarder le résultata
out_path = "result_road_obstacle.jpg"
cv2.imwrite(out_path, img)

print("Image sauvegardée :", out_path)


=== Détections ===
- garbage (0.39)
Image sauvegardée : result_road_obstacle.jpg
